In [1]:
from numpy.random import seed
import numpy as np
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
# from tensorflow.python.keras.optimizers import Fire
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers

from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import matplotlib.pyplot as plt
%matplotlib inline

### CrossEntropyLoss
The input comes from a softmax

        output /= tf.reduce_sum(output,
                                reduction_indices=len(output.get_shape()) - 1,
                                keep_dims=True)
        # manual computation of crossentropy
        epsilon = _to_tensor(_EPSILON, output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon, 1. - epsilon)
        return - tf.reduce_sum(target * tf.log(output),
                               reduction_indices=len(output.get_shape()) - 1)


### Softmax
softmax = tf.exp(logits) / tf.reduce_sum(tf.exp(logits), axis)


## Preparing the input data


In [2]:
#Preparing the input data

#loading the MNIST dataset
mnist = tf.keras.datasets.mnist

#Separating into train and test (60000 train, 10000 test)
(x_train, y_train0),(x_test, y_test0) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#flattening the images (from 28x28 to 784)
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

#converting the otputs (labels) into one hot vectors
lb = preprocessing.LabelBinarizer()
lb.fit(np.array([0,1,2,3,4,5,6,7,8,9]))
y_train = lb.transform(y_train0)
y_test = lb.transform(y_test0)

## Defining the input parameters


In [3]:
#Defining the input parameters
BatchSize         = 6000
NeuronsLayer1     = 100
NeuronsLayer2     = 100
Epochs            = 100

## Model in tensorflow/Keras


In [4]:
# Defining the net

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(NeuronsLayer1,
                        activation=tf.nn.relu),
  tf.keras.layers.Dense(NeuronsLayer2, 
                        activation=tf.nn.relu),
  #  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, 
                        activation=tf.nn.softmax)
                                   ])
print(model)

In [5]:
# Construct our model, # Construct our loss function and an Optimizer. 

loss = tf.keras.backend.categorical_crossentropy
optimizer = optimizers.Adam()

model.compile(optimizer= optimizer,
              loss=loss ) #'categorical_crossentropy')

In [6]:
#train the network

HistoryKeras = model.fit(  x_train, y_train, 
            epochs=Epochs,
            batch_size=BatchSize)


Epoch 1/100
60000/60000 [==============================] - 3s 45us/step - loss: 1.9558
Epoch 2/100
60000/60000 [==============================] - 2s 32us/step - loss: 1.0886
Epoch 3/100
60000/60000 [==============================] - 2s 27us/step - loss: 0.5647
Epoch 4/100
60000/60000 [==============================] - 2s 27us/step - loss: 0.3966
Epoch 5/100
60000/60000 [==============================] - 2s 26us/step - loss: 0.3321
Epoch 6/100
60000/60000 [==============================] - 2s 30us/step - loss: 0.2940
Epoch 7/100
60000/60000 [==============================] - 2s 29us/step - loss: 0.2681
Epoch 8/100
60000/60000 [==============================] - 2s 29us/step - loss: 0.2479
Epoch 9/100
60000/60000 [==============================] - 2s 30us/step - loss: 0.2318
Epoch 10/100
60000/60000 [==============================] - 2s 28us/step - loss: 0.2173
Epoch 11/100
60000/60000 [==============================] - 2s 34us/step - loss: 0.2053
Epoch 12/100
60000/60000 [===============

In [7]:
#Predicting

#model.evaluate(x_test, y_test)
y_train_hat_keras = model.predict_proba(x_train)

log_loss(y_train, y_train_hat_keras)

0.0117205965601813

In [8]:
print(y_train[0])
print(y_train[1])

[0 0 0 0 0 1 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]


In [9]:
print(y_train_hat_keras[0])
print(y_train_hat_keras[1])

[3.9198899e-14 8.0459966e-13 1.3278649e-11 7.2586811e-03 3.9743648e-22
 9.9274135e-01 8.8209048e-16 2.1307798e-11 6.5751636e-12 8.5500045e-12]
[9.9999833e-01 4.2240376e-13 1.7022567e-06 2.8614241e-11 1.3861602e-13
 2.6523841e-14 3.2699721e-09 2.7456237e-11 3.1001646e-14 4.3051358e-11]


In [10]:
#Show aprameters of the network

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  78500     
_________________________________________________________________
dense_1 (Dense)              multiple                  10100     
_________________________________________________________________
dense_2 (Dense)              multiple                  1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________
